In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def graficar(df: pd.DataFrame, feature, linea_cero=False):
    fig, ax = plt.subplots(figsize=[10,5])
    ax.plot(df[feature])
    if linea_cero:
        ax.axhline(y=0,color='grey',linestyle='--')

# Carga de datos
Se importa los datasets:
* JSTdatasetR5 de [http://data.macrohistory.net/JST/JSTdatasetR5.dta](http://data.macrohistory.net/JST/JSTdatasetR5.dta)
* WBdataset extraido de [https://databank.bancomundial.org/](https://databank.bancomundial.org/)

## JSTdatasetR5

In [3]:
df_jst = pd.read_stata("../data/JSTdatasetR5.dta")

In [4]:
df_jst.head()

,year,country,iso,ifs,pop,rgdpmad,rgdppc,rconpc,gdp,iy,...,eq_capgain,eq_dp,eq_capgain_interp,eq_tr_interp,eq_dp_interp,bond_rate,eq_div_rtn,capital_tr,risky_tr,safe_tr
0,1870.0,Australia,AUS,193,1775.0,3273.239437,13.836157,21.449734,208.78,0.109266,...,-0.070045,0.071417,NaN,NaN,NaN,0.049118,0.066415,NaN,NaN,NaN
1,1871.0,Australia,AUS,193,1675.0,3298.507463,13.936864,19.930801,211.56,0.104579,...,0.041654,0.065466,NaN,NaN,NaN,0.048446,0.068193,NaN,NaN,NaN
2,1872.0,Australia,AUS,193,1722.0,3553.426249,15.044247,21.085006,227.40,0.130438,...,0.108945,0.062997,NaN,NaN,NaN,0.047373,0.069861,NaN,NaN,NaN
3,1873.0,Australia,AUS,193,1769.0,3823.629169,16.219443,23.254910,266.54,0.124986,...,0.083086,0.064484,NaN,NaN,NaN,0.046720,0.069842,NaN,NaN,NaN
4,1874.0,Australia,AUS,193,1822.0,3834.796926,16.268228,23.458050,287.58,0.141960,...,0.119389,0.063503,NaN,NaN,NaN,0.046533,0.071085,NaN,NaN,NaN


In [5]:
df_jst.tail()

,year,country,iso,ifs,pop,rgdpmad,rgdppc,rconpc,gdp,iy,...,eq_capgain,eq_dp,eq_capgain_interp,eq_tr_interp,eq_dp_interp,bond_rate,eq_div_rtn,capital_tr,risky_tr,safe_tr
2659,2013.0,USA,USA,111,315820.328999,31571.993947,103.425299,101.892671,16784.851,0.192086,...,0.271035,0.019355,NaN,NaN,NaN,0.023508,0.024601,0.139843,0.212405,-0.065168
2660,2014.0,USA,USA,111,318106.646578,32113.618881,105.186253,104.113597,17527.258,0.196377,...,0.136350,0.019199,NaN,NaN,NaN,0.025408,0.021817,0.130063,0.132729,0.122398
2661,2015.0,USA,USA,111,320413.930388,32800.923063,107.421590,107.192931,18224.780,0.198301,...,-0.000092,0.021124,NaN,NaN,NaN,0.021358,0.021122,0.046193,0.065433,-0.008779
2662,2016.0,USA,USA,111,322705.239927,33078.508719,108.318698,109.333457,18715.040,0.195831,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2663,2017.0,USA,USA,111,324802.861426,33593.446309,110.013284,111.389150,19519.424,0.204547,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Países

In [6]:
df_jst.country.unique()

array(['Australia', 'Belgium', 'Canada', 'Switzerland', 'Germany',
       'Denmark', 'Spain', 'Finland', 'France', 'UK', 'Ireland', 'Italy',
       'Japan', 'Netherlands', 'Norway', 'Portugal', 'Sweden', 'USA'],
      dtype=object)

In [7]:
df_jst.iso.unique()

array(['AUS', 'BEL', 'CAN', 'CHE', 'DEU', 'DNK', 'ESP', 'FIN', 'FRA',
       'GBR', 'IRL', 'ITA', 'JPN', 'NLD', 'NOR', 'PRT', 'SWE', 'USA'],
      dtype=object)

In [8]:
df_jst.isna().sum().sort_values()

year                           0
country                        0
iso                            0
ifs                            0
peg_type                       0
peg_base                       0
crisisJST                     50
peg                           50
peg_strict                    50
pop                           51
cpi                           52
rgdpmad                       52
rgdppc                        52
xrusd                         53
gdp                           77
ltrate                        87
imports                      115
exports                      115
narrowm                      115
crisisJST_old                148
expenditure                  155
rconpc                       156
revenue                      174
stir                         198
money                        201
debtgdp                      229
ca                           229
tloans                       250
iy                           293
lev                          362
bond_rate 

### Preprocesado

In [9]:
features = np.array(['year','iso','pop','rgdpmad','ca','imports',
'exports','revenue','expenditure','debtgdp','narrowm','stir','ltrate','cpi'])

In [10]:
def preprocesarJST(df : pd.DataFrame):
    paises = df.iso.unique()
    df_r = pd.DataFrame()
    df.interpolate()
    for pais in paises:
        df_n = df[df.iso==pais][features].copy()
        df_n = df_n.astype({'year': np.int16})
        #Fija year como indice
        df_n = df_n.set_index('year')
        #Calcula el PBI real usando el PBI per capita y la poblacion
        df_n['rgdp'] = df_n['rgdpmad']*df_n['pop']
        #calcula el ratio de crecimiento del PBI real
        df_n['rgdp_growth'] = df_n['rgdp'].pct_change()
        #interpola datos faltantes en la serie
        df_n = df_n.interpolate(method='quadratic')
        #elimina filas con datos faltantes en los extremos
        df_n = df_n.dropna()

        df_r = df_r.append(df_n)
    return df_r

In [11]:
df_jst = preprocesarJST(df_jst)

In [12]:
df_jst.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2330 entries, 1902 to 2017
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   iso          2330 non-null   object 
 1   pop          2330 non-null   float64
 2   rgdpmad      2330 non-null   float64
 3   ca           2330 non-null   float64
 4   imports      2330 non-null   float64
 5   exports      2330 non-null   float64
 6   revenue      2330 non-null   float64
 7   expenditure  2330 non-null   float64
 8   debtgdp      2330 non-null   float64
 9   narrowm      2330 non-null   float64
 10  stir         2330 non-null   float64
 11  ltrate       2330 non-null   float64
 12  cpi          2330 non-null   float64
 13  rgdp         2330 non-null   float64
 14  rgdp_growth  2330 non-null   float64
dtypes: float64(14), object(1)
memory usage: 291.2+ KB


## WBdataset

In [18]:
df_wb = pd.read_csv('../data/WBdataset.csv')

In [19]:
df_wb.head()

,país Code,tiempo Code,NY.GDP.MKTP.KD,NE.TRD.GNFS.ZS,NE.EXP.GNFS.ZS,NE.GDI.FTOT.ZS,NE.CON.GOVT.ZS,NE.IMP.GNFS.ZS,NV.IND.TOTL.ZS,NV.IND.MANF.ZS,FR.INR.RINR,FP.CPI.TOTL,FM.LBL.BMNY.GD.ZS,FM.LBL.BMNY.ZG,SL.UEM.TOTL.NE.ZS,SL.UEM.TOTL.ZS,SL.EMP.TOTL.SP.NE.ZS,SL.EMP.TOTL.SP.ZS
0,ARG,YR1960,1.507978e+11,15.208096,7.604049,23.478768,9.880445,7.604047,NaN,NaN,NaN,NaN,21.139251,NaN,NaN,NaN,NaN,NaN
1,ARG,YR1961,1.589829e+11,11.989892,5.994947,27.765895,7.789632,5.994945,NaN,NaN,NaN,NaN,16.605999,-0.359712,NaN,NaN,NaN,NaN
2,ARG,YR1962,1.576283e+11,14.075526,4.691843,21.730503,12.192375,9.383683,NaN,NaN,NaN,NaN,14.310117,10.108303,NaN,NaN,NaN,NaN
3,ARG,YR1963,1.492611e+11,15.780906,7.890454,18.272516,10.252195,7.890452,NaN,NaN,NaN,NaN,16.017618,33.114754,NaN,NaN,NaN,NaN
4,ARG,YR1964,1.643817e+11,11.127430,5.563716,17.178900,7.229026,5.563714,NaN,NaN,NaN,NaN,15.967860,41.379310,NaN,NaN,NaN,NaN


In [20]:
df_wb.tail()

,país Code,tiempo Code,NY.GDP.MKTP.KD,NE.TRD.GNFS.ZS,NE.EXP.GNFS.ZS,NE.GDI.FTOT.ZS,NE.CON.GOVT.ZS,NE.IMP.GNFS.ZS,NV.IND.TOTL.ZS,NV.IND.MANF.ZS,FR.INR.RINR,FP.CPI.TOTL,FM.LBL.BMNY.GD.ZS,FM.LBL.BMNY.ZG,SL.UEM.TOTL.NE.ZS,SL.UEM.TOTL.ZS,SL.EMP.TOTL.SP.NE.ZS,SL.EMP.TOTL.SP.ZS
3839,GBR,YR2017,3.088030e+12,61.354757,30.045686,18.018943,18.529638,31.309071,17.829685,8.947602,NaN,114.943592,144.271087,8.296332,4.33,4.330,60.180000,60.186001
3840,GBR,YR2018,3.139011e+12,62.314223,30.506397,17.773526,18.352082,31.807826,17.906024,8.846982,NaN,117.579064,146.520893,5.299826,4.00,4.000,60.560001,60.556999
3841,GBR,YR2019,3.191493e+12,62.927269,31.004978,17.718796,18.800567,31.922291,17.981247,8.879728,NaN,119.622711,139.337724,-1.364158,3.74,3.740,60.849998,60.855999
3842,GBR,YR2020,2.891616e+12,56.108854,28.120834,17.106203,22.347195,27.988020,17.024908,8.648937,NaN,120.806362,160.150856,9.706440,NaN,4.472,NaN,59.991001
3843,GBR,YR2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,123.848715,NaN,NaN,NaN,4.526,NaN,59.653999
